In [1]:
context.catalog.list()

['express_expense',
 'homography_points',
 'homography_points_segmentation',
 'homography_points_segmentation_batched_hull',
 'homography_points_segmentation_batched_hough',
 'homography_boxes',
 'homography_boxes_segmentation',
 'homography_boxes_segmentation_batched',
 'mask',
 'masks',
 'masks_batched',
 'parameters']

In [2]:
points_hull_table = context.catalog.load("homography_points_segmentation_batched_hull")
points_hough_table = context.catalog.load("homography_points_segmentation_batched_hough")
original_images = context.catalog.load("express_expense")

[05/29/23 08:28:44] INFO     Loading data from 'homography_points_segmentation_batched_hull'    ]8;id=470694;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=599491;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (CSVDataSet)...                                                                       

                    INFO     Loading data from 'homography_points_segmentation_batched_hough'   ]8;id=548258;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=695277;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             (CSVDataSet)...                                                                       

                    INFO     Loading data from 'express_expense' (PartitionedDataSet)...        ]8;id=441020;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=500032;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [3]:
import numpy as np
import pandas as pd
import PIL
import re
import cv2

In [4]:
points_hull_table

,index,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y,mask_index
0,batched_1000mask0,181,205,652,181,467,917,118,999,0
1,batched_1000mask1,181,198,662,178,709,999,111,999,1
2,batched_1000mask2,26,173,745,185,747,999,3,999,2
3,batched_1001mask0,296,491,318,487,326,503,298,504,0
4,batched_1001mask1,296,490,319,487,326,503,297,504,1
...,...,...,...,...,...,...,...,...,...,...
457,batched_1197mask1,25,117,286,101,309,449,21,436,1
458,batched_1197mask2,25,117,313,13,310,449,23,439,2
459,batched_1198mask2,6,0,220,13,232,449,1,449,2
460,batched_1199mask1,71,0,278,1,309,347,38,347,1


In [5]:
#for image_number in points_hull["index"].str.replace("batched_", "").str.replace("mask.*", "", regex=True).unique():
loaded_images = dict()
for file_name in original_images:
    image = np.array(original_images[file_name]())
    loaded_images[file_name] = image

In [6]:
area_ratios = list()
for i, (index, p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y, mask_index) in points_hull_table.iterrows():
    img_number = int(re.sub("mask.*", "", index.replace("batched_", "")))
    original_image = loaded_images[str(img_number) + "-receipt.jpg"]
    img_area = original_image.shape[0] * original_image.shape[1]
    points = np.array([[p1x, p1y], [p2x, p2y], [p3x, p3y], [p4x, p4y]])
    points_area = cv2.contourArea(points)
    area_ratios.append(points_area / img_area)
    
points_hull_table["area_ratio"] = area_ratios

In [7]:
points_hull_table

,index,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y,mask_index,area_ratio
0,batched_1000mask0,181,205,652,181,467,917,118,999,0,0.409437
1,batched_1000mask1,181,198,662,178,709,999,111,999,1,0.583226
2,batched_1000mask2,26,173,745,185,747,999,3,999,2,0.799857
3,batched_1001mask0,296,491,318,487,326,503,298,504,0,0.000581
4,batched_1001mask1,296,490,319,487,326,503,297,504,1,0.000617
...,...,...,...,...,...,...,...,...,...,...,...
457,batched_1197mask1,25,117,286,101,309,449,21,436,1,0.601972
458,batched_1197mask2,25,117,313,13,310,449,23,439,2,0.715615
459,batched_1198mask2,6,0,220,13,232,449,1,449,2,0.892821
460,batched_1199mask1,71,0,278,1,309,347,38,347,1,0.683825


In [8]:
points_hull_table["area_ratio"].gt(0.35).value_counts(normalize=True)

True     0.839827
False    0.160173
Name: area_ratio, dtype: float64

In [9]:
points_hull_table = points_hull_table[points_hull_table["area_ratio"] > 0.35].reset_index(drop=True)

In [10]:
points_hull_table

,index,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y,mask_index,area_ratio
0,batched_1000mask0,181,205,652,181,467,917,118,999,0,0.409437
1,batched_1000mask1,181,198,662,178,709,999,111,999,1,0.583226
2,batched_1000mask2,26,173,745,185,747,999,3,999,2,0.799857
3,batched_1001mask2,89,49,551,80,617,902,71,948,2,0.672670
4,batched_1002mask1,83,25,415,181,370,999,2,999,1,0.424685
...,...,...,...,...,...,...,...,...,...,...,...
383,batched_1197mask1,25,117,286,101,309,449,21,436,1,0.601972
384,batched_1197mask2,25,117,313,13,310,449,23,439,2,0.715615
385,batched_1198mask2,6,0,220,13,232,449,1,449,2,0.892821
386,batched_1199mask1,71,0,278,1,309,347,38,347,1,0.683825


In [11]:
points_hough_table

,index,P1X,P1Y,P2X,P2Y,P3X,P3Y,P4X,P4Y,mask_index
0,batched_1000mask0,690,958,658,181,114,958,173,198,0
1,batched_1000mask1,665,178,108,997,172,195,715,997,1
2,batched_1001mask2,76,952,545,74,64,45,569,905,2
3,batched_1002mask1,362,997,79,28,23,997,446,113,1
4,batched_1002mask2,371,997,26,997,456,997,524,997,2
...,...,...,...,...,...,...,...,...,...,...
386,batched_1197mask2,288,96,293,93,288,93,293,96,2
387,batched_1198mask2,233,447,0,447,215,447,227,447,2
388,batched_1199mask0,304,345,58,162,290,160,39,345,0
389,batched_1199mask1,77,0,37,345,310,345,277,0,1


In [12]:
points_hough_table = points_hough_table.set_index("index")
points_hull_table = points_hull_table.set_index("index")

In [13]:
points_hough_table = points_hough_table[points_hough_table.index.isin(points_hull_table.index)]
points_hull_table = points_hull_table[points_hull_table.index.isin(points_hough_table.index)]

In [14]:
points_hull = list()
for i, (p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y, mask_index, area_ratio) in points_hull_table.iterrows():
    points_hull.append(np.array([[p1x, p1y], [p2x, p2y], [p3x, p3y], [p4x, p4y]]).astype(np.int32))

In [15]:
points_hough = list()
for i, (p1x, p1y, p2x, p2y, p3x, p3y, p4x, p4y, mask_index) in points_hough_table.iterrows():
    points_hough.append(np.array([[p1x, p1y], [p2x, p2y], [p3x, p3y], [p4x, p4y]]).astype(np.int32))

In [17]:
#from sibigrapi2023.pipelines.processing.nodes import _calc_iou

In [72]:
def _calc_iou(ground_truth, pred):
    gtxs = ground_truth[:, 0]
    gtys = ground_truth[:, 1]
    prxs = pred[:, 0]
    prys = pred[:, 1]

    ix1 = np.concatenate([gtxs, prxs]).min()
    ix2 = np.concatenate([gtxs, prxs]).max()
    iy1 = np.concatenate([gtys, prys]).min()
    iy2 = np.concatenate([gtys, prys]).max()
     
    # Intersection height and width.
    i_height = np.maximum(abs(iy2 - iy1) + 1, 0)
    i_width = np.maximum(abs(ix2 - ix1) + 1, 0)
     
    area_of_intersection = i_height * i_width
     
    # Ground Truth dimensions.
    gt_area = cv2.contourArea(ground_truth)
    pr_area = cv2.contourArea(pred)
     
    area_of_union = gt_area + pr_area - area_of_intersection
     
    iou = area_of_intersection / area_of_union
     
    return iou

In [73]:
ious = list()
for i, point in enumerate(points_hull):
    ious.append(_calc_iou(point, points_hough[i]))

In [74]:
ious = pd.Series(ious)

In [75]:
ious.gt(0).value_counts()

False    263
True      78
dtype: int64